# Análisis de resultados de varias carteras y posibilidad de representación gráfica

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
from sklearn.metrics import confusion_matrix, classification_report

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import recall_score
from sklearn.metrics import make_scorer

from sklearn.neighbors import KNeighborsClassifier

import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv("../tablas/dataformodel.csv", usecols=['Price_d', 'Price_d+180',
                                                        'quantile_PER', 'var_quantile_PER','PER','var_PER',
                                                        'quantile_PBC', 'var_quantile_PBC','var_PBC',
                                                        'quantile_ROA', 'var_quantile_ROA','var_ROA',
                                                        'quantile_Dot_dudosos', 'var_quantile_Dot_dudosos',
                                                        'Dot_dudosos', 'var_Dot_dudosos',
                                                        'Etiqueta', 'Periodo','Ticker'])

df=df.replace([np.inf, -np.inf], np.nan)
for column in df.columns:
    df=df[df[column].notnull()]
df=df.reset_index(drop=True)

In [3]:
def resultado(row):
    if row['Prediction'] == 'Peor':
        return (row['Price_d'] - row['Price_d+180'])/row['Price_d']
    else:
        return (row['Price_d+180'] - row['Price_d'])/row['Price_d']

In [13]:
lista_periodos=df.Periodo.unique()[20:21]
titulos = 17
inversion = 100
resumen = pd.DataFrame(columns=['Periodo','Rentabilidad'])
ratios = ['quantile_PER','var_quantile_PER','quantile_PBC','var_quantile_PBC']

for trim_seleccionado in lista_periodos:
    df_train = df[df.Periodo<trim_seleccionado]
    df_test = df[df.Periodo==trim_seleccionado]
    
    X_train=df_train[ratios].values
    
    y_train=pd.Series(df_train['Etiqueta'].values)
    
    
    X_test=df_test[ratios].values
    
    y_test=df_test['Etiqueta'].values
    
    clfk=KNeighborsClassifier(n_neighbors=5)
    clfk.fit(X_train,y_train)
    
    knGrid = GridSearchCV(clfk,cv=5,scoring="accuracy",param_grid={'n_neighbors':np.arange(1,20)})
    knGrid.fit(X_train,y_train)
    best_param=knGrid.best_params_.get('n_neighbors')
    
    clfk=KNeighborsClassifier(n_neighbors=best_param)
    clfk.fit(X_train,y_train)
    
    df_prob=pd.DataFrame({'Probabilidad':clfk.predict_proba(X_test)[:,0],'Prediction':clfk.predict(X_test),'Actual':y_test})
    df_prob.index=df_test.index

    best=df_prob.sort_values('Probabilidad').head(titulos)
    worst=df_prob.sort_values('Probabilidad').tail(titulos)
    cartera=best.append(worst)
    
    cartera = cartera.join(df_test)
    res_cartera=cartera[['Prediction','Price_d','Price_d+180']]
    
    res_cartera['Resultado']=res_cartera.apply(resultado,axis=1)*inversion
    
    
    resumen = resumen.append({'Periodo':trim_seleccionado,'Rentabilidad':res_cartera.Resultado.mean()}, ignore_index=True)
    

df_var_indice = pd.read_csv("../tablas/var_indice.csv")
resumen = resumen.merge(df_var_indice, 
                              left_on=('Periodo'),
                              right_on=('Periodo'), 
                              how='left')

resumen.drop(['Indice_d_100','Indice_d+180_100'], axis=1, inplace=True)

resumen
    

,Periodo,Rentabilidad,Rent_indice
0,2016Q3,-4.010067,25.447536


In [11]:
df_var_indice

,Periodo,Indice_d_100,Indice_d+180_100,Rent_indice
0,2011Q2,85.446712,97.377288,13.962593
1,2011Q3,86.162284,102.377520,18.819412
2,2011Q4,97.365754,101.211930,3.950236
3,2012Q1,102.206891,104.503845,2.247357
4,2012Q2,100.426559,112.954443,12.474672
5,2012Q3,102.816106,118.758031,15.505280
6,2012Q4,114.211571,135.881108,18.973153
7,2013Q1,119.325598,143.346036,20.130163
8,2013Q2,139.435976,145.906951,4.640822
9,2013Q3,144.344565,148.789265,3.079229


In [10]:
resumen.Rentabilidad.sum()

2.4404737577080926

In [8]:
resumen.Rent_indice.sum()

20.26276498465987

In [14]:
cartera

,Probabilidad,Prediction,Actual,Ticker,Periodo,Price_d,Price_d+180,Etiqueta,PER,Dot_dudosos,...,var_ROA,var_Dot_dudosos,quantile_PER,quantile_PBC,quantile_ROA,quantile_Dot_dudosos,var_quantile_PER,var_quantile_PBC,var_quantile_ROA,var_quantile_Dot_dudosos
20,0.8,Mejor,Peor,C,2016Q3,44.67,54.21,Peor,33.501337,-0.000955,...,0.040771,-0.188656,8.823529,2.941176,32.352941,2.941176,5.882353,0.000000,-14.705882,-5.882353
49,0.2,Peor,Peor,CFR,2016Q3,67.01,82.95,Peor,53.511396,-0.000170,...,-0.092538,0.860799,58.823529,79.411765,67.647059,85.294118,-11.764706,0.000000,11.764706,5.882353
76,0.4,Peor,Mejor,CIT,2016Q3,33.76,42.25,Mejor,51.868710,-0.000684,...,-0.872147,-0.489062,52.941176,5.882353,23.529412,8.823529,-44.117647,0.000000,20.588235,-58.823529
76,0.4,Peor,Mejor,CIT,2016Q3,33.76,42.25,Mejor,51.868710,-0.000684,...,-0.872147,-0.489062,52.941176,5.882353,23.529412,8.823529,-44.117647,0.000000,20.588235,-58.823529
106,0.6,Mejor,Mejor,CMA,2016Q3,46.51,63.53,Mejor,53.737905,-0.000216,...,-0.276287,2.184691,61.764706,58.823529,26.470588,79.411765,-29.411765,14.705882,20.588235,58.823529
135,0.4,Peor,Mejor,EWBC,2016Q3,35.97,51.82,Mejor,47.066707,-0.000286,...,-0.053649,-0.358670,38.235294,85.294118,88.235294,64.705882,0.000000,0.000000,0.000000,-26.470588
165,0.6,Mejor,Peor,FITB,2016Q3,20.37,22.37,Peor,30.530029,-0.000558,...,-0.392684,0.134760,2.941176,35.294118,91.176471,26.470588,-29.411765,20.588235,52.941176,0.000000
165,0.6,Mejor,Peor,FITB,2016Q3,20.37,22.37,Peor,30.530029,-0.000558,...,-0.392684,0.134760,2.941176,35.294118,91.176471,26.470588,-29.411765,20.588235,52.941176,0.000000
195,0.4,Peor,Mejor,FMBH,2016Q3,25.15,32.05,Mejor,49.736504,-0.000388,...,0.151503,-0.109471,44.117647,38.235294,20.588235,50.000000,14.705882,-17.647059,-23.529412,-20.588235
224,0.8,Mejor,Peor,FNB,2016Q3,11.01,12.29,Peor,46.163688,-0.000678,...,-0.203053,0.156458,35.294118,32.352941,47.058824,11.764706,-32.352941,0.000000,26.470588,5.882353


In [15]:
res_cartera

,Prediction,Price_d,Price_d+180,Resultado
20,Mejor,44.67,54.21,21.356615
49,Peor,67.01,82.95,-23.787494
76,Peor,33.76,42.25,-25.148104
76,Peor,33.76,42.25,-25.148104
106,Mejor,46.51,63.53,36.594281
135,Peor,35.97,51.82,-44.064498
165,Mejor,20.37,22.37,9.818360
165,Mejor,20.37,22.37,9.818360
195,Peor,25.15,32.05,-27.435388
224,Mejor,11.01,12.29,11.625795
